In [ ]:
import geopandas as gpd
import pandas as pd
import time
import os
import numpy as np
import geemap
import ee
import json
from tqdm import tqdm
import concurrent.futures

# from google.colab import auth , drive
# drive.mount('/content/drive') Only required if you are using google colab
from helper_functions import  get_shadow_index_for_month, get_ndvi_for_month

In [ ]:
ee.Authenticate()
ee.Initialize(project='test-reforestation')

In [ ]:
### Assuming this is the combined dataset and has  previously gee
df = gpd.read_parquet("../midsave/consolidated_reforestation_projects.parquet")
df.info()

In [ ]:
gdf = df.loc[:,['site_id_created', 'geometry']]

In [ ]:
gdf.dropna(subset=['geometry'], inplace = True)

In [ ]:
chunk_size = 1000
chunks = [gdf[i:i + chunk_size] for i in range(0, gdf.shape[0], chunk_size)]

In [ ]:
S2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')

In [ ]:
output_csv_path = '../midsave/buffer_ndvi.csv'
ndvi = pd.DataFrame()

### NDVI Buffer

In [ ]:


for i, chunk in enumerate(chunks):
    print(f"Processing chunk {i + 1}/{len(chunks)}")

    # Creating a 500-meter buffer around each feature
    chunk['geometry'] = chunk['geometry'].buffer(500)

    gdf_json_chunk = chunk.__geo_interface__

    try:
        fc_chunk = geemap.geojson_to_ee(gdf_json_chunk)
    except Exception as e:
        print(f"Error converting chunk {i + 1} to Earth Engine FeatureCollection: {e}")
        continue

    ndvi_chunk = fc_chunk.map(lambda feature: get_ndvi_for_month(feature, S2))

    temp_chunk_df = pd.DataFrame([feature['properties'] for feature in ndvi_chunk.getInfo()['features']])

    # Appending to combined DataFrame
    ndvi = pd.concat([ndvi, temp_chunk_df], ignore_index=True)

buffer_ndvi_grouped = ndvi.groupby(['site_id_created'])['ndvi'].mean().reset_index()
buffer_ndvi_grouped.to_csv(output_csv_path, index=False)

print("All chunks processed, combined results saved to:", output_csv_path)

## Shadow Index Buffer

In [ ]:
gdf_si = (gdf
       .merge(df[['site_id_created', 'planting_date_reported', 'month', 'ndvi_monthly_mean']], on = 'site_id_created', how = 'left')
       .reset_index(drop = True))
gdf_si['planting_date_reported'] = gdf_si['planting_date_reported'].astype(int)
gdf_si['month'] = gdf_si['month'].astype(int)

In [ ]:
chunk_size = 1000
chunks = [gdf_si[i:i + chunk_size] for i in range(0, gdf_si.shape[0], chunk_size)]

In [ ]:
output_csv_path = '../midsave/buffer_shadow_index.csv'
shadow_index = pd.DataFrame()

In [ ]:

for i, chunk in enumerate(chunks):
    print(f"Processing chunk {i + 1}/{len(chunks)}")

    # Creating a 500-meter buffer around each feature
    chunk['geometry'] = chunk['geometry'].buffer(500)

    gdf_json_chunk = chunk.__geo_interface__

    try:
        fc_chunk = geemap.geojson_to_ee(gdf_json_chunk)
    except Exception as e:
        print(f"Error converting chunk {i + 1} to Earth Engine FeatureCollection: {e}")
        continue

    shadow_index_chunk = fc_chunk.map(lambda feature: get_shadow_index_for_month(feature, S2))

    temp_chunk_df = pd.DataFrame([feature['properties'] for feature in shadow_index_chunk.getInfo()['features']])

    # Appending to combined DataFrame
    shadow_index = pd.concat([shadow_index, temp_chunk_df], ignore_index=True)

buffer_shadow_index_grouped = shadow_index.groupby(['site_id_created'])['shadow_index'].mean().reset_index()
buffer_shadow_index_grouped.to_csv(output_csv_path, index=False)

print("All chunks processed, combined results saved to:", output_csv_path)

In [ ]:
df_combined = (df.merge(buffer_shadow_index_grouped, on = 'site_id_created', how = 'left')
               .merge(buffer_ndvi_grouped, on = 'site_id_created', how = 'left'))

In [ ]:
df_combined.info()